# MD Simulation of Alkanes

https://education.molssi.org/mm-tools/02-md-alkanes/index.html

In [1]:
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit

In [2]:
pdb = app.PDBFile('ethane.pdb')
forcefield = app.ForceField('ethane.gaff2.xml')

In [3]:
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

In [4]:
integrator = mm.LangevinIntegrator(298.15*unit.kelvin, 5.0/unit.picoseconds, 2.0*unit.femtoseconds)
integrator.setConstraintTolerance(1e-5)

In [5]:
platform = mm.Platform.getPlatformByName('Reference')
simulation = app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)

## Energy Minimization

In [6]:
print('Minimizing...')

st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy before minimization is {st.getPotentialEnergy()}")

simulation.minimizeEnergy(maxIterations=100)

st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy after minimization is {st.getPotentialEnergy()}")

Minimizing...
Potential energy before minimization is 4.467818271065188 kJ/mol
Potential energy after minimization is 4.389967724953272 kJ/mol


## Equilibration

In [7]:
from sys import stdout

print('Equilibrating...')

simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True, 
    potentialEnergy=True, temperature=True, separator=','))
simulation.context.setVelocitiesToTemperature(150.0*unit.kelvin)
simulation.step(2500)

Equilibrating...
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
100,15.002682493596097,236.97807372828973
200,20.709687222785394,165.29999849353277
300,17.39773530259589,230.71020822751893
400,30.035605547501127,215.18444541491198
500,18.483058303579966,323.6497906036022
600,15.380448167856343,475.0101901928745
700,19.640984269573465,418.80508787781923
800,22.737471271888953,343.71292399528613
900,18.342068345523902,376.5354888659024
1000,14.589877743870643,181.28903541622827
1100,13.281095823767355,184.37763557085577
1200,37.42013050667864,184.5703975220423
1300,9.524734796454633,392.25466965157966
1400,22.102155012595354,261.44923840819916
1500,12.941140346725641,243.9174982181316
1600,20.015576526822755,405.49473507900956
1700,13.456924248147173,159.83047690023852
1800,10.400336615854801,304.1264438875253
1900,17.338944025163737,373.375721311773
2000,19.73728126602726,164.30938401878007
2100,24.112479612703957,210.219336725836
2200,26.948462123049985,292.21913167359173
2300,

## Production

In [8]:
import time as time

print('Running Production...')

# Begin timer
tinit=time.time()

# Clear simulation reporters
simulation.reporters.clear()

# Reinitialize simulation reporters. We do this because we want different information printed from the production run than the equilibration run.
# output basic simulation information below every 250000 steps - (which is equal to 2 fs(250,000) = 500,000 fs = 500 ps)
simulation.reporters.append(app.StateDataReporter(stdout, 250000, 
    step=True, time=True, potentialEnergy=True, temperature=True, 
    speed=True, separator=','))

# write out a trajectory (i.e., coordinates vs. time) to a DCD
# file every 100 steps - 0.2 ps
simulation.reporters.append(app.DCDReporter('ethane_sim.dcd', 100))

# run the simulation for 1.0x10^7 steps - 20 ns
simulation.step(10000000)

# End timer
tfinal=time.time()
print('Done!')
print('Time required for simulation:', tfinal-tinit, 'seconds')

Running Production...
#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
250000,500.0000000016593,26.660765190536335,150.95196266983973,0
500000,999.9999999901769,23.36795876010662,269.0180991465003,1.56e+04
750000,1499.9999999783536,16.278367478345857,337.22275326798496,1.57e+04
1000000,1999.9999999665301,20.679593342562605,395.972225003714,1.56e+04
1250000,2499.9999999547067,15.021243259055439,187.44633442358705,1.51e+04
1500000,2999.9999999428833,27.74217446551597,242.76051980315157,1.49e+04
1750000,3499.99999993106,18.74136540597931,420.4945171784268,1.5e+04
2000000,3999.9999999192364,13.671880964327647,368.6080215639798,1.5e+04
2250000,4499.9999999992715,25.75737069589573,450.2522438684371,1.5e+04
2500000,5000.000000101135,14.937842173510212,298.8869766250064,1.48e+04
2750000,5500.000000202998,15.642919870766871,428.4362374098017,1.49e+04
3000000,6000.000000304862,18.94368299734819,195.9678053554878,1.5e+04
3250000,6500.000000406725,16.955719289707

## Analysis

In [9]:
simulation.reporters.append(app.DCDReporter('ethane_sim.dcd', 100))

In [10]:
import mdtraj as md

traj = md.load('ethane_sim.dcd', top='ethane.pdb')

ModuleNotFoundError: No module named 'mdtraj'

In [ ]:
import nglview as ngl

visualize = ngl.show_mdtraj(traj)
visualize

In [ ]:
atoms, bonds = traj.topology.to_dataframe()
atoms

## Analyzing the C-C bond length

In [ ]:
bond_indices = [0, 4] # atoms to define the bond length
bond_length = md.compute_distances(traj, [bond_indices])

In [ ]:
import matplotlib.pyplot as plt

bondcounts, binedges, otherstuff = plt.hist(bond_length, bins=120)
plt.title('C-C bond length histogram')
plt.xlabel('Bond length (nm)')
plt.ylabel('Counts')
plt.show()